OUTLINE

MODULE 1: Data preprocessing and loading

(a) Obtaining and cleaning the datasets

(b) Twitter specific text pre-processing

(c) Word / sentence vectorisation (as first data input)

Michael: BERT and FastText

(d) Implementing a dictionary approach potentially based on Hatebase.org (as second data input)
(e) Splitting data into test, validation and testing set
(f) Specifying and implementing the data loader
(g) Testing and iterating the module
(h) Creatingmodule-specificvisualisationsforthefinal paper

MODULE 2: Model architecture and training

(a) Choosing width and depth of the model
(b) Choosingoptimizeraswellasactivationandloss functions
(c) Choosing stopping rule, regularisation, dropout, learning rates etc.
(d) Potentially performing a hyperparameter grid search
(e) Running and tracking the training
(f) Training and tuning the model based on the re- sults of the validation set
(g) Creatingmodule-specificvisualisationsforthefi- nal paper

In [12]:
import pandas as pd

In [13]:
# (a) Obtaining and cleaning the datasets
url_davidson = "https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/data/twitter%20data/davidson%20et%20al./labeled_data.csv"
url_founta = "https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/data/twitter%20data/founta%20et%20al./hatespeech_text_label_vote.csv"


data_davidson = pd.read_csv(url_davidson, names=["count", "hate_speech", "offensive_language", "neither", "label", "tweet"], header=1)
data_founta = pd.read_csv(url_founta, sep='\t', names=["tweet", "label_text", "count"])

data_founta = data_founta[~data_founta.label_text.str.contains("spam")]
data_founta['label'] = data_founta.label_text.replace({'hateful': '0', 'abusive': '1', 'normal': '2'}).astype('int')
data_founta.label.hist()

# concatinating and shuffling


HTTPError: HTTP Error 404: Not Found

In [ ]:
data_davidson.label.hist()

In [ ]:
import pandas as pd
df = pd.read_csv("/Users/mxm/Google Drive/Masterstudium/Inhalte/4th Semester/NLP/nlp-project/coding/code/exchange_base/data.csv")
df.label.hist()

In [ ]:
# trying to re-import hatebase, somehow doesn't work

'''
import json 
import requests
import pandas as pd
from hatebase import HatebaseAPI

key = open('C:/Users/Niko/Documents/NJS Cloud/1 Uni (non-Google)/0 Hertie/2 Spring 2020/NLP/Hatebase_Key', "r").read() # insert filepath where the API key is stored
filepath = "C:/Users/Niko/Documents/NJS Cloud/1 Uni (non-Google)/0 Hertie/2 Spring 2020/NLP/full_dictionary.csv" # insert filepath where the final csv file should be stored 

hatebase = HatebaseAPI({"key": key})
filters = {"language": "eng"}
format = "json"
# initialize list for all vocabulary entry dictionaries
eng_vocab = []
response = hatebase.getVocabulary(filters=filters, format=format)
pages = response["number_of_pages"]
# fill the vocabulary list with all entries of all pages
# this might take some time...
for page in range(1, pages+1):
    filters["page"] = str(page) 
    response = hatebase.getVocabulary(filters=filters, format=format)
    eng_vocab.append(response["result"])

data = []
for r in eng_vocab:
    data.append(r["data"])
#print len(data)
listofHatewords = []

#print len(data)
for z in data:
    for a, v in z.iteritems():
        for b in v:
            listofHatewords.append(b["vocabulary"])
# print listofHatewords
listofHatewords = list(OrderedDict.fromkeys(listofHatewords))
print(len(listofHatewords))
'''


In [1]:
import csv
import sys
import nltk
import pprint
import pandas as pd
import torch

#Loading Hatebase-Data from CSV-File

hatebase_dic = pd.read_csv('C:/Users/Niko/Documents/GitHub/nlp-project/nlp-project-1/coding/data/dictionary/hatebase/full_dictionary.csv')

#pprint.pprint(hatebase_dic)

# Getting Twitter data
input = "https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/code/exchange_base/data.csv"
data = pd.read_csv(input)

#pprint.pprint(data)


In [2]:
# Lemmatizing both the dictionary and the tweets

import spacy
#import dframcy

# loading the spacy model
nlp = spacy.load("en_core_web_sm")
# switching off irrelevant spacy functions
nlp.disable_pipes('tagger', 'ner')

# lemmatizing the dictionary
lemma = []
for term in nlp.pipe(hatebase_dic['term'].astype('unicode').values, batch_size=50, n_threads=3):
    if term.is_parsed:
        lemma.append([n.lemma_ for n in term])
    else:
        lemma.append(None)

# storing the lemmas in a new column 
hatebase_dic['term_lemma'] = lemma

# storing the lemmas in a new column 
hatebase_dic['term_lemma'] = lemma

#making lemmas lowercase
hatebase_dic['term_lemma'] = hatebase_dic['term_lemma'].map(lambda lemmas: [x.lower() for x in lemmas])
    
#joining list of lemmas into one string agein, so it becomes matchable by the searchfunction
hatebase_dic['term_lemma'] = hatebase_dic['term_lemma'].map(lambda lemmas: " ".join(lemmas))

# lemmatizing the tweets
lemma = []
for tweet in nlp.pipe(data['tweet'].astype('unicode').values, batch_size=50, n_threads=3):
    if tweet.is_parsed:
        lemma.append([n.lemma_ for n in tweet])
    else:
        lemma.append(None)

# storing the lemmas in a new column 
data['tweet_lemma'] = lemma

#making lemmas lowercase
data['tweet_lemma'] = df['tweet_lemma'].map(lambda lemmas: [x.lower() for x in lemmas])
    
#joining list of lemmas into one string agein, so it becomes matchable by the searchfunction
data['tweet_lemma'] = df['tweet_lemma'].map(lambda lemmas: " ".join(lemmas))

NameError: name 'df' is not defined

In [4]:
# Search function

import pandarellel
pandarallel.initialize() 

def hatesearch(data):
    frequency = 0
    hatefulness = 0
    for term_lemma in hatebase_dic:
        if term_lemma in data['tweet_lemma']:
            frequency += 1
            hatefulness = hatebase_dic[hatebase_dic['term_lemma'] == term_lemma].average_offensiveness
    data['Hatefreq'] = frequency
    data['Hatefulness'] = hatefulness
    return data

data = data.parallel_apply(hatesearch, axis = 1)

#data = data.apply(lambda x : hatesearch, axis=1)

#print(data)

# Exporting to browse through

data.to_csv('C:/Users/Niko/Documents/newdata.csv')

ModuleNotFoundError: No module named 'pandarellel'

In [3]:
# Building a tensor out of the additional columns

HateFrequency = torch.tensor(data['Hatefreq'])
torch.save(HateFrequency, 'https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/code/exchange_base/HateFreq_Tensor.pt')

SyntaxError: invalid syntax (<ipython-input-3-c2f23e36ac51>, line 3)